In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gp
from sklearn.preprocessing import normalize
from scipy.sparse import coo_matrix, csr_matrix


%matplotlib inline

In [4]:
tops=pd.read_csv(r'D:\jonahfull.csv')
tops.ssdepth.min()

-11183

In [5]:
tops.ssdepth = tops.ssdepth-tops.ssdepth.min()

In [29]:
tops.head()

,api,formation,depth,ssdepth,normd
0,3521913,WASATCH,0,18464,0
1,3521913,LANCE,9285,9179,-4.633233533
2,3521913,LANCE - OVER PRESSURE,9760,8704,-3.9370714
3,3521913,JONAH,11227,7237,-2.845159655
4,3521913,YELLOW POINT,11793,6671,-2.613696809


In [7]:
data=np.asarray(tops[['api', 'formation', 'ssdepth']])
tops['depth'].max()

18245

In [8]:
D_df = tops.pivot_table('ssdepth', 'formation', 'api').fillna(0)

In [9]:
R =D_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [20]:
print('Number of data points: {}'.format(n_wells*n_formations))

Number of data points: 1264562


In [21]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 10)
sigma = np.diag(sigma)


In [22]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = D_df.columns, index=D_df.index)

In [11]:
from sklearn.preprocessing import binarize
A=binarize(R)

In [30]:
def runALS(A, R, n_factors, n_iterations, lambda_):
    '''
    Runs Alternating Least Squares algorithm in order to calculate matrix.
    :param A: User-Item Matrix with ratings
    :param R: User-Item Matrix with 1 if there is a rating or 0 if not
    :param n_factors: How many factors each of user and item matrix will consider
    :param n_iterations: How many times to run algorithm
    :param lambda_: Regularization parameter
    :return:
    '''
    print("Initiating ")
    lambda_ = 0.1; n_factors = 20; n, m = A.shape; n_iterations = 30
    Users = 5 * np.random.rand(n, n_factors)
    Items = 5 * np.random.rand(n_factors, m)

    def get_error(A, Users, Items, R):
        # This calculates the MSE of nonzero elements
        return np.sum((R * (A - np.dot(Users, Items))) ** 2) / np.sum(R)

    MSE_List = []

    print("Starting Iterations")
    for iter in range(n_iterations):
        for i, Ri in enumerate(R):
            Users[i] = np.linalg.solve(np.dot(Items, np.dot(np.diag(Ri), Items.T)) + lambda_ * np.eye(n_factors),
                                       np.dot(Items, np.dot(np.diag(Ri), A[i].T))).T
        print("Error after solving for User Matrix:", get_error(A, Users, Items, R))

        for j, Rj in enumerate(R.T):
            Items[:,j] = np.linalg.solve(np.dot(Users.T, np.dot(np.diag(Rj), Users)) + lambda_ * np.eye(n_factors),
                                     np.dot(Users.T, np.dot(np.diag(Rj), A[:, j])))
        print("Error after solving for Item Matrix:", get_error(A, Users, Items, R))

        MSE_List.append(get_error(A, Users, Items, R))
        print('%sth iteration is complete...' % iter)
    return Users
    print(MSE_List)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(range(1, len(MSE_List) + 1), MSE_List); plt.ylabel('Error'); plt.xlabel('Iteration')
    plt.title('Python Implementation MSE by Iteration \n with %d formations and %d wells' % A.shape);
    #plt.savefig('Python MSE Graph.pdf', format='pdf')
    plt.show()

In [ ]:
runALS(R, A, 10, 100, 0.1)

Initiating 
Starting Iterations
Error after solving for User Matrix: 2900758.0064716153


In [ ]:
Users